# Import libraries

In [ ]:
import geotrellis.raster._
import geotrellis.raster.vectorize._
import geotrellis.raster.vectorize
import geotrellis.raster.io.geotiff._
import geotrellis.raster.render._
import geotrellis.raster.io.geotiff.GeoTiff

import geotrellis.spark._
import geotrellis.spark.io.RasterReader
import geotrellis.spark.io.hadoop._
import geotrellis.spark.io.hadoop
import geotrellis.spark.tiling.FloatingLayoutScheme

import geotrellis.vector._

import org.apache.spark.rdd._
import org.apache.spark.SparkContext
import org.apache.spark.mllib.linalg.Vectors

import java.net.URI 
import scala.math.BigDecimal.RoundingMode

import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.fs.Path

import com.mongodb.spark._
import com.mongodb.spark.config._
import org.bson.Document

import geotrellis.raster._
import geotrellis.raster.vectorize._
import geotrellis.raster.vectorize
import geotrellis.raster.io.geotiff._
import geotrellis.raster.render._
import geotrellis.raster.io.geotiff.GeoTiff
import geotrellis.spark._
import geotrellis.spark.io.RasterReader
import geotrellis.spark.io.hadoop._
import geotrellis.spark.io.hadoop
import geotrellis.spark.tiling.FloatingLayoutScheme
import geotrellis.vector._
import org.apache.spark.rdd._
import org.apache.spark.SparkContext
import org.apache.spark.mllib.linalg.Vectors
import java.net.URI
import scala.math.BigDecimal.RoundingMode
import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.fs.Path
import com.mongodb.spark._
import com.mongodb.spark.config._
import org.bson.Document


# Important variables 

In [ ]:
// implicit variable (important variables to run the functions in the geotrellis library)
implicit val sparkContext = sc

val rr = implicitly[RasterReader[HadoopGeoTiffRDD.Options, (ProjectedExtent, Tile)]]

sparkContext: org.apache.spark.SparkContext = org.apache.spark.SparkContext@7135b49
rr: geotrellis.spark.io.RasterReader[geotrellis.spark.io.hadoop.HadoopGeoTiffRDD.Options,(geotrellis.vector.ProjectedExtent, geotrellis.raster.Tile)] = geotrellis.spark.io.RasterReader$$anon$1@4535b758


In [ ]:
val options =
HadoopGeoTiffRDD.Options(
  numPartitions = Some(100)
)

options: geotrellis.spark.io.hadoop.HadoopGeoTiffRDD.Options = Options(List(.tif, .TIF, .tiff, .TIFF),None,TIFFTAG_DATETIME,yyyy:MM:dd HH:mm:ss,Some(256),Some(100),Some(134217728),None)


# Parameters

In [ ]:
val HdfsUrl = "hdfs://hupi-factory-02-01-01-01/"
val dataRepo = "user/factory02/thailand_workshop/ndvi/"
val landsatName = "LT05_L1GS_125052_20070915_20161112_01_T2"
// LT05_L1GS_125052_20070915_20161112_01_T2 
// LC08_L1TP_125052_20171231_20180103_01_T1

HdfsUrl: String = hdfs://hupi-factory-02-01-01-01/
dataRepo: String = user/factory02/thailand_workshop/ndvi/
landsatName: String = LT05_L1GS_125052_20070915_20161112_01_T2


In [ ]:
val saveName = HdfsUrl + dataRepo + landsatName + "_withNDVIMean"

saveName: String = hdfs://hupi-factory-02-01-01-01/user/factory02/thailand_workshop/ndvi/LT05_L1GS_125052_20070915_20161112_01_T2_withNDVIMean


# Delete the PNG if existed in HDFS

In [ ]:
val conf = sc.hadoopConfiguration 

conf: org.apache.hadoop.conf.Configuration = Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml


In [ ]:
val fs = org.apache.hadoop.fs.FileSystem.get(new java.net.URI(HdfsUrl), conf)

fs: org.apache.hadoop.fs.FileSystem = DFS[DFSClient[clientName=DFSClient_NONMAPREDUCE_-943843208_12, ugi=root (auth:SIMPLE)]]


In [ ]:
// Remove the image png of NDVI if it's there already
fs.delete(new Path(saveName + ".png"),true)

res8: Boolean = true


true

# Read geotiff NDVI

In [ ]:
val ndvi = HadoopGeoTiffRDD[ProjectedExtent, Tile](
      new Path(HdfsUrl + dataRepo + landsatName + ".tif"), 
      options).map(l => (l._1, l._2.convert(DoubleConstantNoDataCellType)))        

ndvi: org.apache.spark.rdd.RDD[(geotrellis.vector.ProjectedExtent, geotrellis.raster.Tile)] = MapPartitionsRDD[4] at map at <console>:119


In [ ]:
// Convert Tile to ArrayDouble
val arrayNDVI = ndvi.map(l => (l._1.extent, l._2.toArrayDouble))

arrayNDVI: org.apache.spark.rdd.RDD[(geotrellis.vector.Extent, Array[Double])] = MapPartitionsRDD[5] at map at <console>:120


# Summary Statistics of NDVI by Tile

By Tile, we compute value min, max, mean, standard deviation of NDVI

In [ ]:
val name_landsat = landsatName
// We replace NaN by NODATA = -2147483648 and compute descriptive stats by tile
val summaryStatsByTile = arrayNDVI.map {
  case (extent, arrayDouble) => {
    // Length of array of each Tile
    val lengthArray = arrayDouble.length
    // If array contains NaN => we transform whole array to array of NODATA
    if (arrayDouble.filter(l => l.isNaN).length > 1) {
      val newArray = Array.fill(lengthArray)(NODATA.toDouble)
      val mean = newArray.sum / lengthArray
      // we return new RDD of 10 columns : xmin, ymin, xmax, ymax, ...and summary stats of NDVI per Tile
      (name_landsat, extent.xmin, extent.ymin, extent.xmax, extent.ymax, extent.width, extent.height,
       newArray.min, newArray.max,
       newArray.sum / lengthArray, 
       Math.sqrt((newArray.map( _ - mean).map(t => t*t).sum)/lengthArray))
    } else {
      val mean = arrayDouble.sum / lengthArray
      (name_landsat, extent.xmin, extent.ymin, extent.xmax, extent.ymax, extent.width, extent.height,
       arrayDouble.min, arrayDouble.max,
       mean, 
       Math.sqrt((arrayDouble.map( _ - mean).map(t => t*t).sum)/lengthArray))
    }
  }
}

name_landsat: String = LT05_L1GS_125052_20070915_20161112_01_T2
summaryStatsByTile: org.apache.spark.rdd.RDD[(String, Double, Double, Double, Double, Double, Double, Double, Double, Double, Double)] = MapPartitionsRDD[6] at map at <console>:123


# Prepare output to save in MongoDB

In [ ]:
// Convert RDD to Dataframe
val df = summaryStatsByTile.toDF("landsatName", "xmin", "ymin", "xmax", "ymax", "width", "height", "minNDVI", "maxNDVI", "meanNDVI", "stdDevNDVI")

df: org.apache.spark.sql.DataFrame = [landsatName: string, xmin: double ... 9 more fields]


In [ ]:
/*
// We save the collection to Mongo by overwriting it (if it's old image and appending if it's new image)
df.write.format("com.mongodb.spark.sql").option("uri", s"mongodb://10.100.2.7:27017/hupi.descriptive_stats_NDVI")
.mode("overwrite").save()
*/

To check if data is there in Mongo, we can connect directly to the server Mongo : 10.100.2.7 then run command : "mongo 10.100.2.7/hupi" --> show collections 

# Visualization Results

We compute NDVI mean for each Tile then for each pixel in this Tile, we replace its NDVI by meanNDVI and combine all into one raster then render to PNG

In [ ]:
// For each Tile in ndvi, we compute meanNDVI and replace all NDVI in this Tile by meanNDVI
val ndviFinal = ndvi.map {
  case (pe, tile) => {
    val arrayTile = tile.toArrayDouble
    val lengthArrayTile = arrayTile.length
    val meanNDVI = arrayTile.sum / lengthArrayTile
    (pe, DoubleArrayTile(Array.fill(lengthArrayTile)(meanNDVI), tile.cols, tile.rows).tile)
  }
}

ndviFinal: org.apache.spark.rdd.RDD[(geotrellis.vector.ProjectedExtent, geotrellis.raster.Tile)] = MapPartitionsRDD[7] at map at <console>:120


In [ ]:
// We create color map
val ndviColormap = "0:ffffe5ff;0.1:f7fcb9ff;0.2:d9f0a3ff;0.3:addd8eff;0.4:78c679ff;0.5:41ab5dff;0.6:238443ff;0.7:006837ff;1:004529ff"

// Get color map from the application.conf settings file.
val colorMap = ColorMap.fromStringDouble(ndviColormap).get

ndviColormap: String = 0:ffffe5ff;0.1:f7fcb9ff;0.2:d9f0a3ff;0.3:addd8eff;0.4:78c679ff;0.5:41ab5dff;0.6:238443ff;0.7:006837ff;1:004529ff
colorMap: geotrellis.raster.render.ColorMap = geotrellis.raster.render.DoubleColorMap@7a9f7cc1


In [ ]:
// Get metadata and tiles to convert to raster
val (_, metadata) = ndviFinal.collectMetadata[SpatialKey](FloatingLayoutScheme())
val tiles = ndviFinal.tileToLayout[SpatialKey](metadata)

metadata: geotrellis.spark.TileLayerMetadata[geotrellis.spark.SpatialKey] = TileLayerMetadata(float64,GridExtent(Extent(527085.0, 1167075.0, 765165.0, 1382115.0),30.0,30.0),Extent(527085.0, 1167075.0, 765165.0, 1382115.0),EPSG:32648,KeyBounds(SpatialKey(0,0),SpatialKey(30,27)))
tiles: org.apache.spark.rdd.RDD[(geotrellis.spark.SpatialKey, geotrellis.raster.Tile)] = ShuffledRDD[10] at reduceByKey at TileRDDMerge.scala:51


In [ ]:
// We create Raster by stitching all tiles 
val raster_ndvi = ContextRDD(tiles, metadata).stitch

raster_ndvi: geotrellis.raster.Raster[geotrellis.raster.Tile] = Raster(DoubleConstantNoDataArrayTile([D@758c939,7936,7168),Extent(527085.0, 1167075.0, 765165.0, 1382115.0))


# Render to PNG

In [ ]:
val ndvi_png = raster_ndvi.tile.renderPng(colorMap)

ndvi_png: geotrellis.raster.render.Png = Png([B@6574866e)


# Save PNG to HDFS

In [ ]:
ndvi_png.write(new Path(saveName + ".png"))

No codec found for hdfs://hupi-factory-02-01-01-01/user/factory02/thailand_workshop/ndvi/LT05_L1GS_125052_20070915_20161112_01_T2_withNDVIMean.png, writing without compression.


# Save PNG to server

In [ ]:
val ndviPath_server = "/opt/docker/notebooks/data/" + landsatName + "_withNDVIMean.png"

ndviPath_server: String = /opt/docker/notebooks/data/LT05_L1GS_125052_20070915_20161112_01_T2_withNDVIMean.png


In [ ]:
ndvi_png.write(ndviPath_server)

# Print image in notebook

In [ ]:
import java.io.File

import java.io.File


In [ ]:
val image_ndvi = img() // default type and size
image_ndvi.file(new File(ndviPath_server))

image_ndvi: notebook.front.SingleConnectedWidget[java.awt.image.BufferedImage]{implicit val codec: notebook.Codec[play.api.libs.json.JsValue,java.awt.image.BufferedImage]; lazy val toHtml: scala.xml.Elem; def url(u: java.net.URL): Unit; def file(f: java.io.File): Unit} = <$anon$1 widget>


In [ ]:
image_ndvi

res29: notebook.front.SingleConnectedWidget[java.awt.image.BufferedImage]{implicit val codec: notebook.Codec[play.api.libs.json.JsValue,java.awt.image.BufferedImage]; lazy val toHtml: scala.xml.Elem; def url(u: java.net.URL): Unit; def file(f: java.io.File): Unit} = <$anon$1 widget>
